In [18]:
from seqdataloader.seqdataloader.labelgen import *

In [24]:
![[ -f ENCFF289XSX.bigWig ]] || wget https://www.encodeproject.org/files/ENCFF289XSX/@@download/ENCFF289XSX.bigWig
# !ln -s ENCFF071ZMW.bed.gz peaks_with_signal_SPI1_new.bed.gz

/bin/sh: 1: [[: not found
--2019-08-25 02:02:02--  https://www.encodeproject.org/files/ENCFF289XSX/@@download/ENCFF289XSX.bigWig
Resolving www.encodeproject.org (www.encodeproject.org)... 34.211.244.144
Connecting to www.encodeproject.org (www.encodeproject.org)|34.211.244.144|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://download.encodeproject.org/https://encode-public.s3.amazonaws.com/2019/02/13/9a778a35-5a1a-4b1d-9207-bd2f09af2d07/ENCFF289XSX.bigWig?response-content-disposition=attachment%3B%20filename%3DENCFF289XSX.bigWig&x-amz-security-token=AgoJb3JpZ2luX2VjEID%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLXdlc3QtMiJHMEUCIHCKEh%2FJRbBQ0MaEMnH9%2BJS7STRqu2w%2BJHa7UfLynCzaAiEA8t%2FAq%2BiKitvohB3g2uLz%2B9C8%2FF1MPd%2F30EQHLOttqasq2gMIKRAAGgwyMjA3NDg3MTQ4NjMiDN3S8DqM%2FOL82yZQjSq3A38F%2B2ZdModYXtXFeZSoGYugZmFvKbJ%2BUtlTdviTdBLrUBT3TLGH1LvqIDIzzk8hvq5CwOIyV2r65nXsOlnWJf6bIruu9pv%2BozbDWDb9uBLMmbsbLq8%2BUIc20Qz6AKjRP7xV%2BH%2Bnyc4oGWlqcXWgAhN

In [30]:
! zcat peaks_with_signal.bed.gz | perl -lane 'print $F[0]."\t".($F[1]+$F[9])."\t".($F[1]+$F[9])."\t+\t".($F[6])' | egrep -w 'chr1|chr2|chr3|chr4|chr5|chr6|chr7|chr8|chr9|chr10|chr11|chr12|chr13|chr14|chr15|chr16|chr17|chr18|chr19|chr20|chr21|chr22|chrX|chrY' | gzip -c > summits_with_signal.bed.gz

In [31]:
! zcat regressionlabels.tsv.gz | perl -lane 'print $F[0]."\t".($F[1]+$F[9])."\t".($F[1]+$F[9])."\t+\t".($F[6])' | egrep -w 'chr1|chr2|chr3|chr4|chr5|chr6|chr7|chr8|chr9|chr10|chr11|chr12|chr13|chr14|chr15|chr16|chr17|chr18|chr19|chr20|chr21|chr22|chrX|chrY' | gzip -c > summits_with_signal_SPI1_new.bed.gz

#We split into training/test/validation set by chromosome
!zcat summits_with_signal_SPI1_new.bed.gz | egrep -w 'chr1|chr8|chr21' | gzip -c > test_summits_with_signal_SPI1_new.bed.gz
!zcat summits_with_signal_SPI1_new.bed.gz | egrep -w 'chr22' | gzip -c > valid_summits_with_signal_SPI1_new.bed.gz
!zcat summits_with_signal_SPI1_new.bed.gz | egrep -w -v 'chr1|chr8|chr21|chr22' | gzip -c > train_summits_with_signal_SPI1_new.bed.gz

In [1]:
from seqdataloader.seqdataloader.batchproducers import coordbased
import gzip
import numpy as np

class ColsInBedFile(
    coordbased.coordstovals.core.AbstractSingleNdarrayCoordsToVals):
    def __init__(self, gzipped_bed_file, **kwargs):
        super(ColsInBedFile, self).__init__(**kwargs)
        self.gzipped_bed_file = gzipped_bed_file
        coords_to_vals = {}
        for row in gzip.open(gzipped_bed_file, 'rb'):
            row = row.decode("utf-8").rstrip()
            split_row = row.split("\t")
            chrom_start_end = split_row[0]+":"+split_row[1]+"-"+split_row[2]
            vals = np.array([float(x) for x in split_row[4:]])
            coords_to_vals[chrom_start_end] = vals
        self.coords_to_vals = coords_to_vals
        
    def _get_ndarray(self, coors):
        to_return = []
        for coor in coors:
            chrom_start_end = (coor.chrom+":"
                               +str(coor.start)+"-"+str(coor.end))
            to_return.append(self.coords_to_vals[chrom_start_end])
        return np.array(to_return)
    
    
inputs_coordstovals = coordbased.coordstovals.fasta.PyfaidxCoordsToVals(
  genome_fasta_path= '/mnt/data/annotations/by_release/hg38/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta',
  center_size_to_use=1000)

targets_coordstovals = ColsInBedFile(
       gzipped_bed_file="summits_with_signal_SPI1_new.bed.gz")
            
keras_train_batch_generator = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer=coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
      bed_file="train_summits_with_signal_SPI1_new.bed.gz",
      #coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
      batch_size=64,
      shuffle_before_epoch=True,
      seed=1234
    ),
    inputs_coordstovals=inputs_coordstovals,
    targets_coordstovals=targets_coordstovals
)


keras_valid_batch_generator = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer = coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
        bed_file="valid_summits_with_signal_SPI1_new.bed.gz", 
        batch_size=64, 
        shuffle_before_epoch=True, 
        seed=1234
    ),
    inputs_coordstovals=inputs_coordstovals, 
    targets_coordstovals=targets_coordstovals
)

keras_test_batch_generator = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer = coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
        bed_file="test_summits_with_signal_SPI1_new.bed.gz", 
        batch_size = 64, 
        shuffle_before_epoch = True, 
        seed = 1234
    ), 
    inputs_coordstovals = inputs_coordstovals, 
    targets_coordstovals = targets_coordstovals
)


keras_train_batch_generator_augment = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer=coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
      bed_file="train_summits_with_signal_SPI1_new.bed.gz",
      coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
      batch_size=128,
      shuffle_before_epoch=True,
      seed=1234
    ),
    inputs_coordstovals=inputs_coordstovals,
    targets_coordstovals=targets_coordstovals
)


keras_valid_batch_generator_augment = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer = coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
        bed_file="valid_summits_with_signal_SPI1_new.bed.gz",
        coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
        batch_size=128, 
        shuffle_before_epoch=True, 
        seed=1234
    ),
    inputs_coordstovals=inputs_coordstovals, 
    targets_coordstovals=targets_coordstovals
)

keras_test_batch_generator_augment = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer = coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
        bed_file="test_summits_with_signal_SPI1_new.bed.gz",
        coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
        batch_size = 128, 
        shuffle_before_epoch = True, 
        seed = 1234
    ), 
    inputs_coordstovals = inputs_coordstovals, 
    targets_coordstovals = targets_coordstovals
)

Using TensorFlow backend.


Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to yo

In [2]:
inputs_coordstovals.ltrdict = {
           'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1],
           'n':[0,0,0,0],'A':[1,0,0,0],'C':[0,1,0,0],'G':[0,0,1,0],
           'T':[0,0,0,1],'N':[0,0,0,0],'R': [0.5,0,0.5,0],'Y':[0,0.5,0,0.5]}

In [ ]:
y_test = np.array([val for batch in keras_test_batch_generator for val in batch[1]], dtype = 'float32') 

In [ ]:
y_test_augment = np.array([val for batch in keras_test_batch_generator_augment for val in batch[1]], dtype = 'float32') 

In [ ]:
import keras 
import keras_genomics
import numpy as np
import keras.layers as k1
import simdna

from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.layers import Input
from keras.engine import Layer
from keras.models import Sequential 
from keras.engine.base_layer import InputSpec
from keras.models import Model
from keras.models import load_model

In [ ]:
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import random_ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import ops
import numbers
from tensorflow.python.framework import tensor_util
def _get_noise_shape(x, noise_shape):
  # If noise_shape is none return immediately.
  if noise_shape is None:
    return array_ops.shape(x)

  try:
    # Best effort to figure out the intended shape.
    # If not possible, let the op to handle it.
    # In eager mode exception will show up.
    noise_shape_ = tensor_shape.as_shape(noise_shape)
  except (TypeError, ValueError):
    return noise_shape

  if x.shape.dims is not None and len(x.shape.dims) == len(noise_shape_.dims):
    new_dims = []
    for i, dim in enumerate(x.shape.dims):
      if noise_shape_.dims[i].value is None and dim.value is not None:
        new_dims.append(dim.value)
      else:
        new_dims.append(noise_shape_.dims[i].value)
    return tensor_shape.TensorShape(new_dims)

  return noise_shape

class MCRCDropout(Layer):
    """Applies MC Dropout to the input.
       The applied noise vector is symmetric to reverse complement symmetry
       Class structure only slightly adapted 
    Dropout consists in randomly setting
    a fraction `rate` of input units to 0 at each update during training time,
    which helps prevent overfitting.
    Remains active ative at test time so sampling is required
    # Arguments
        rate: float between 0 and 1. Fraction of the input units to drop.
        noise_shape: 1D integer tensor representing the shape of the
            binary dropout mask that will be multiplied with the input.
            For instance, if your inputs have shape
            `(batch_size, timesteps, features)` and
            you want the dropout mask to be the same for all timesteps,
            you can use `noise_shape=(batch_size, 1, features)`.
        seed: A Python integer to use as random seed.
    # References
        - [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf)
    """
    def __init__(self, rate, noise_shape=None, seed=None, **kwargs):
        super(MCRCDropout, self).__init__(**kwargs)
        self.rate = min(1., max(0., rate))
        self.noise_shape = noise_shape
        self.seed = seed
        self.supports_masking = True
        
    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(MCRCDropout, self).build(input_shape)

    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape

        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                       for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)

    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            import numpy as np
            noise_shape = self._get_noise_shape(inputs)
            x = inputs
            seed = self.seed
            keep_prob = 1. - self.rate
            if seed is None:
                seed = np.random.randint(10e6)
            # the dummy 1. works around a TF bug
            # (float32_ref vs. float32 incompatibility)
            x= x*1
            name = None
            with ops.name_scope(name, "dropout", [x]) as name:
                x = ops.convert_to_tensor(x, name="x")
                if not x.dtype.is_floating:
                    raise ValueError("x has to be a floating point tensor since it's going to"
                       " be scaled. Got a %s tensor instead." % x.dtype)
                if isinstance(keep_prob, numbers.Real) and not 0 < keep_prob <= 1:
                    raise ValueError("keep_prob must be a scalar tensor or a float in the "
                       "range (0, 1], got %g" % keep_prob)
                keep_prob = ops.convert_to_tensor(
                             keep_prob, dtype=x.dtype, name="keep_prob")
                keep_prob.get_shape().assert_is_compatible_with(tensor_shape.scalar())

                # Do nothing if we know keep_prob == 1
                if tensor_util.constant_value(keep_prob) == 1:
                    return x

                noise_shape = _get_noise_shape(x, noise_shape)
                # uniform [keep_prob, 1.0 + keep_prob)
                random_tensor = keep_prob
                random_tensor += random_ops.random_uniform(
                noise_shape, seed=seed, dtype=x.dtype)
               
                # 0. if [keep_prob, 1.0) and 1. if [1.0, 1.0 + keep_prob)
                binary_tensor = math_ops.floor(random_tensor)
                dim = binary_tensor.shape[2]//2

                symmetric_binary = K.concatenate(
                    tensors = [
                      binary_tensor[:,:,int(self.num_input_chan/2):], 
                      binary_tensor[:,:,int(self.num_input_chan/2):][::,::-1,::-1]], 
                  axis=2)
                ret = math_ops.div(x, keep_prob) * symmetric_binary
                
                return ret


    def get_config(self):
        config = {'rate': self.rate,
                  'noise_shape': self.noise_shape,
                  'seed': self.seed}
        base_config = super(MCRCDropout, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

In [ ]:
class RevCompSpatialDropout1D(Dropout): 
    def __init__(self, rate,**kwargs): 
        super(RevCompSpatialDropout1D, self).__init__(rate, **kwargs)
        self.seed = 3
        self.input_spec = InputSpec(ndim = 3)

    def _get_noise_shape(self, inputs): 
        input_shape = K.shape(inputs)
        noise_shape = (input_shape[0], 1, 1, int(self.num_input_chan/2)) 
        return noise_shape
        
    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        self.input_len = input_shape[1]
        super(RevCompSpatialDropout1D, self).build(input_shape)

    def call(self, inputs, training=None): 
        inputs_fwdandrevconcat = K.concatenate(
                tensors = [
                    inputs[:,:,None,:int(self.num_input_chan/2)],
                    inputs[:,:,None,int(self.num_input_chan/2):][:,:,:,::-1]],
                axis=2)

        if 0. < self.rate < 1.: 
            noise_shape = self._get_noise_shape(inputs)
            def dropped_inputs(): 
                dropped = K.dropout(inputs_fwdandrevconcat,
                                    self.rate, noise_shape, seed = self.seed)
                dropped = K.reshape(dropped, (-1, int(self.input_len), int(self.num_input_chan)))
                return K.concatenate(
                    tensors = [
                        dropped[:,:,:int(self.num_input_chan/2)],
                        dropped[:,:,int(self.num_input_chan/2):][:,:,::-1]],
                    axis=-1)

            return K.in_train_phase(dropped_inputs, inputs, training = training)

        return inputs

In [ ]:
class RevCompSumPool(Layer): 
    def __init__(self, **kwargs): 
        super(RevCompSumPool, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(RevCompSumPool, self).build(input_shape)

    def call(self, inputs): 
        #divide by sqrt 2 for variance preservation
        inputs = (inputs[:,:,:int(self.num_input_chan/2)] + inputs[:,:,int(self.num_input_chan/2):][:,::-1,::-1])/(1.41421356237)
        return inputs
      
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], int(input_shape[2]/2))

In [ ]:
class RevComp(Layer): 
    def __init__(self, **kwargs): 
      super(RevComp, self).__init__(**kwargs)

    def build(self, input_shape):
      super(RevComp, self).build(input_shape)

    def call(self, inputs): 
      return inputs[:,::-1,::-1]
      
    def compute_output_shape(self, input_shape):
      return input_shape

In [ ]:
kernel_size = 15
filters= 15
input_length = 1000

from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, History, ModelCheckpoint

seed_num = 10000
seed(seed_num)
set_random_seed(seed_num)

In [ ]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

In [ ]:
import json
def test(data):
    num_conv = data["num_conv"]
    dataset = "SPI1"
    augment =  data["augment"]
    dropout = data["dropout"]
    dropout_rate = data["dropout_rate"]
    filename = data["filename"]
    filters = data["filters"]
    kernel_size = data["kernel_size"]
    mc_dropout = data["mc_dropout"]
    num_epochs = data["num_epochs"]
    patience = data["patience"]
    pool_size = data["pool_size"]
    pooling = data["pooling"]
    rev_comp = data["rev_comp"]
#     seed_num = (int)(np.random.uniform(1000, 10000))
    data["seed_num"] = seed_num
    siamese = data["siamese"]
    spatial_dropout = data["spatial_dropout"]
    rc_spatial_dropout = data["rc_spatial_dropout"]
    strides = data["strides"]
    units = data["units"]
    
    train_batch_generator = keras_train_batch_generator
    valid_batch_generator = keras_valid_batch_generator
    test_batch_generator = keras_test_batch_generator
    y_test_model = y_test
    

    if augment: 
        train_batch_generator = keras_train_batch_generator_augment
        valid_batch_generator = keras_valid_batch_generator_augment
        test_batch_generator = keras_test_batch_generator_augment
        y_test_model = y_test_augment
        
    seed(seed_num)
    set_random_seed(seed_num)
    model = keras.models.Sequential()
    
    for i in range(num_conv): 
        if rev_comp: 
            model.add(keras_genomics.layers.RevCompConv1D(filters=filters, 
                                                          kernel_size=kernel_size, 
                                                          input_shape=train_batch_generator[0][0].shape[1:], 
                                                          padding="same"))
        else: 
            model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                          input_shape=train_batch_generator[0][0].shape[1:],
                          padding="same")) 
            
        model.add(k1.core.Activation("relu"))
        
        #No dropout before the pooling layer
        if i != num_conv - 1: 
            if dropout: 
                model.add(k1.Dropout(dropout_rate))
            elif spatial_dropout: 
                model.add(k1.SpatialDropout1D(dropout_rate))
            elif rc_spatial_dropout:
                model.add(RevCompSpatialDropout1D(dropout_rate)) 
            elif mc_dropout: 
                model.add(MCRCDropout(dropout_rate))

    #Only needed with rc model
    if rev_comp: 
        model.add(RevCompSumPool())
    
    #Pooling layers
    if pooling == 'max': 
        model.add(k1.pooling.MaxPooling1D(pool_size = pool_size,padding = "same", strides = strides))
    elif pooling == 'avg':
        model.add(k1.pooling.AveragePooling1D(pool_size = pool_size, padding = "same", strides = strides))
    
    model.add(Flatten())
    
    #Fully-connected layers 
    model.add(keras_genomics.layers.core.Dense(units = units, activation = "relu"))
    model.add(keras_genomics.layers.core.Dense(units = 1))  
    
    if siamese: 
        main_input = Input(shape=train_batch_generator[0][0].shape[1:])
        rev_input = Input(shape=train_batch_generator[0][0].shape[1:])
        rev_input = RevComp()(main_input)
        main_output = model(main_input)
        rev_output = model(rev_input)
        avg = k1.Average()([main_output, rev_output])
        model = Model(inputs = main_input, outputs = avg)

    model.summary()
    model.compile(optimizer="adam", loss="mean_squared_error")
    early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor = 'val_loss',
                              patience = patience,
                              restore_best_weights=True)
    y_pred = []
    
    model.fit_generator(generator = train_batch_generator, 
                           epochs = num_epochs, callbacks = [early_stopping_callback],
                           validation_data = valid_batch_generator)
    model.set_weights(early_stopping_callback.best_weights)

        
    model.save("/users/hannahgz/revcomp_experiments/SPI1_New_Results/%s_%s.h5" % (filename,str(seed_num)))

    rho = 0
    if dropout or mc_dropout or spatial_dropout or rc_spatial_dropout: 
        rho = spearmanr_all(y_test_model, model, test_batch_generator)
    else: 
        y_pred = model.predict_generator(test_batch_generator)
        rho, pval = spearmanr(y_test_model, y_pred)
    
    data["correlation"] = rho
    
    with open("/users/hannahgz/revcomp_experiments/SPI1_New_Results/config_%s_%s.json" % (filename,str(seed_num)), "w") as data_file: 
        json.dump(data, data_file, indent=2)

In [ ]:
for seed_num in range(1000, 11000, 1000): 
    data = {
            "filename": "rc_15_filters",
            "dataset": "SPI1",
            "augment": False,
            "siamese": False,
            "rev_comp": True,
            "dropout": False,
            "mc_dropout": False,
            "spatial_dropout": False,
            "rc_spatial_dropout": False,
            "dropout_rate": 0.2,
            "num_conv": 3,
            "filters": 15,
            "kernel_size": 15,
            "pool_size": 40,
            "pooling": "max",
            "num_epochs": 1,
            "patience": 60,
            "seed_num": seed_num,
            "strides": 40,
            "units": 100,
            "correlation": 0 
            }
    test(data)


    data["filename"] = "reg_15_filters"
    data["rev_comp"] = False
    test(data)

    data["filename"] = "augment_15_filters"
    data["augment"] = True
    test(data)

    data["filename"] = "siamese_15_filters"
    data["siamese"] = True
    data["augment"] = False
    test(data)